In [1]:
import requests
import pandas as pd
import numpy as np
import os

In [2]:
import simplejson as json

In [3]:
BEARER_TOKEN = os.environ['BEARER_TOKEN_TW']


## Get User

In [4]:
user_url = "https://api.twitter.com/2/users/by/username/btppubliceye"

payload={}
headers = {
  'Authorization': 'Bearer {}'.format(BEARER_TOKEN),
  
}


In [5]:
user_response = requests.request("GET", user_url, headers=headers, data=payload)
user_id = user_response.json()['data']['id']
print(user_id)

761577241960599552


In [30]:
def get_original_tweet(id):
    try:
        url = "https://api.twitter.com/2/tweets/{}?media.fields=url,media_key&expansions=attachments.media_keys".format(str(id[0]['id']))
        tweet = requests.request("GET", url, headers=headers, data=payload)
        tweet = tweet.json()
        #"{},{}".format()
        text,media = '',''
        if('text' in tweet['data'].keys()):
            text = tweet['data']['text']
        if('media' in tweet['includes']):
            media = tweet['includes']['media'][0]['url']
        return [text,media]
    except:
        return ["",""]

In [31]:
a = get_original_tweet([{'type': 'replied_to', 'id': '1344349264743317505'}])

## User Timeline

In [16]:
df =pd.DataFrame.from_dict({"in_reply_to_user_id":[],"text":[],"created_at":[],"id":[]})

In [17]:
start_date = '2020-07-01T00:00:01Z'
end_date = '2020-12-31T23:59:59Z'
def get_timeline_url(user_id,token):
    timeline_url =''
    if(len(token)>0):
        timeline_url="https://api.twitter.com/2/users/{}/tweets?expansions=attachments.media_keys&tweet.fields=in_reply_to_user_id,text,created_at,referenced_tweets&media.fields=preview_image_url,url&start_time={}&end_time={}&max_results=99".format(user_id,start_date,end_date)
        timeline_url=timeline_url+"&pagination_token={}".format(token)
    else:
        timeline_url="https://api.twitter.com/2/users/{}/tweets?expansions=attachments.media_keys&tweet.fields=in_reply_to_user_id,text,created_at,referenced_tweets&media.fields=preview_image_url,url&start_time={}&end_time={}&max_results=99".format(user_id,start_date,end_date)

    return timeline_url

In [18]:
i=0
token=''
response_json =[]
pagination_token=True
while(pagination_token==True):
    if i==0:
        timeline_url = get_timeline_url(user_id,'')
        timeline_response = requests.request("GET", timeline_url, headers=headers, data=payload)
        response_json.append(timeline_response.json())
        data = pd.json_normalize(response_json[0],record_path=['data'])
        df = pd.concat([df,data])
        if('meta' in timeline_response.json().keys()):
            print(timeline_response.json()['meta'])
            if 'next_token' in timeline_response.json()['meta'].keys():
                token = timeline_response.json()['meta']['next_token']
                i=1
            else:
                pagination_token=False
    else:
        timeline_url=get_timeline_url(user_id,token)
        timeline_response = requests.request("GET", timeline_url, headers=headers, data=payload)
        response_json.append(timeline_response.json())
        data = pd.json_normalize(response_json[-1],record_path=['data'])
        df = pd.concat([df,data])
        if('meta' in timeline_response.json().keys()):
            print(timeline_response.json()['meta'])
            if 'next_token' in timeline_response.json()['meta'].keys():
                token = timeline_response.json()['meta']['next_token']
                i=i+1
            else:
                pagination_token=False
        else:
                pagination_token=False
        print(timeline_response.json()['data'][-1]['created_at'])
           
        

{'oldest_id': '1328591773925568512', 'newest_id': '1344647448392679432', 'result_count': 99, 'next_token': '7140dibdnow9c7btw3w1ujouq8f5dc3rw3meobjo7wz76'}
{'oldest_id': '1311221652705832961', 'newest_id': '1328206911028690945', 'result_count': 99, 'next_token': '7140dibdnow9c7btw3w1ez4x3y7m9noprqzmj1a2arrjl', 'previous_token': '77qpymm88g5h9vqkluldaz6aqbwf2bcdj3zom3upw91kl'}
2020-09-30T08:29:37.000Z
{'oldest_id': '1286542337322049536', 'newest_id': '1311220356221267968', 'result_count': 99, 'next_token': '7140dibdnow9c7btw3t4lh1uo3rayvefqpnhqn3jseq0m', 'previous_token': '77qpymm88g5h9vqklulcven9uy4ucl40fb82h9shdxwzc'}
2020-07-24T06:02:50.000Z
{'oldest_id': '1278560102824374272', 'newest_id': '1286294750941728769', 'result_count': 36, 'previous_token': '77qpymm88g5h9vqkluig1wjb9fbcyvjec45xn39ztk01l'}
2020-07-02T05:24:17.000Z


In [19]:
df.head()

,in_reply_to_user_id,text,created_at,id,referenced_tweets,attachments.media_keys
0,779270730139574272,@SrihariGoutham @rajajinagartrps @Mvmtraffic,2020-12-31T14:11:48.000Z,1344647448392679432,"[{'type': 'replied_to', 'id': '134455832535518...",NaN
1,3911755279,@Chets_Bhandary @JnagarTr @JnagarTr check and ...,2020-12-31T14:10:09.000Z,1344647032481280001,"[{'type': 'replied_to', 'id': '134459997907614...",NaN
2,2472048134,@devda_vishal Please wait. We will check and l...,2020-12-30T13:09:27.000Z,1344269369874923522,"[{'type': 'replied_to', 'id': '134392161145229...",NaN
3,415639796,@shyamkumar0531 @Tejasvi_Surya @PCMohanMP @BBM...,2020-12-30T13:03:36.000Z,1344267894813642754,"[{'type': 'replied_to', 'id': '134419557884710...",NaN
4,779270730139574272,@SrihariGoutham @rajajinagartrps @rajajinagart...,2020-12-30T13:02:25.000Z,1344267599287177216,"[{'type': 'replied_to', 'id': '134426628904952...",NaN


In [33]:
#df=df[df['referenced_tweets'].notna()]
df['original_text']=df['referenced_tweets'].apply(get_original_tweet)

In [34]:
df.count()

in_reply_to_user_id       324
text                      333
created_at                333
id                        333
referenced_tweets         333
attachments.media_keys      8
original_text             333
dtype: int64

In [35]:
df.head()

,in_reply_to_user_id,text,created_at,id,referenced_tweets,attachments.media_keys,original_text
0,779270730139574272,@SrihariGoutham @rajajinagartrps @Mvmtraffic,2020-12-31T14:11:48.000Z,1344647448392679432,"[{'type': 'replied_to', 'id': '134455832535518...",NaN,"[, ]"
1,3911755279,@Chets_Bhandary @JnagarTr @JnagarTr check and ...,2020-12-31T14:10:09.000Z,1344647032481280001,"[{'type': 'replied_to', 'id': '134459997907614...",NaN,"[@JnagarTr @btppubliceye \nSir, tree branches ..."
2,2472048134,@devda_vishal Please wait. We will check and l...,2020-12-30T13:09:27.000Z,1344269369874923522,"[{'type': 'replied_to', 'id': '134392161145229...",NaN,"[, ]"
3,415639796,@shyamkumar0531 @Tejasvi_Surya @PCMohanMP @BBM...,2020-12-30T13:03:36.000Z,1344267894813642754,"[{'type': 'replied_to', 'id': '134419557884710...",NaN,"[, ]"
4,779270730139574272,@SrihariGoutham @rajajinagartrps @rajajinagart...,2020-12-30T13:02:25.000Z,1344267599287177216,"[{'type': 'replied_to', 'id': '134426628904952...",NaN,[@btppubliceye @rajajinagartrps This car was p...


In [36]:
#df = df[df['original_text']!='']
df[['original_tweet','media_url']]= pd.DataFrame(df['original_text'].tolist(),columns=['original_tweet','media_url'])

In [37]:
df.head()

,in_reply_to_user_id,text,created_at,id,referenced_tweets,attachments.media_keys,original_text,original_tweet,media_url
0,779270730139574272,@SrihariGoutham @rajajinagartrps @Mvmtraffic,2020-12-31T14:11:48.000Z,1344647448392679432,"[{'type': 'replied_to', 'id': '134455832535518...",NaN,"[, ]",,
1,3911755279,@Chets_Bhandary @JnagarTr @JnagarTr check and ...,2020-12-31T14:10:09.000Z,1344647032481280001,"[{'type': 'replied_to', 'id': '134459997907614...",NaN,"[@JnagarTr @btppubliceye \nSir, tree branches ...","@JnagarTr @btppubliceye \nSir, tree branches c...",https://pbs.twimg.com/media/Eqj6iisUYAEmdbX.jpg
2,2472048134,@devda_vishal Please wait. We will check and l...,2020-12-30T13:09:27.000Z,1344269369874923522,"[{'type': 'replied_to', 'id': '134392161145229...",NaN,"[, ]",,
3,415639796,@shyamkumar0531 @Tejasvi_Surya @PCMohanMP @BBM...,2020-12-30T13:03:36.000Z,1344267894813642754,"[{'type': 'replied_to', 'id': '134419557884710...",NaN,"[, ]",,
4,779270730139574272,@SrihariGoutham @rajajinagartrps @rajajinagart...,2020-12-30T13:02:25.000Z,1344267599287177216,"[{'type': 'replied_to', 'id': '134426628904952...",NaN,[@btppubliceye @rajajinagartrps This car was p...,@btppubliceye @rajajinagartrps This car was pa...,https://pbs.twimg.com/media/EqfLDMuXYAU9u82.jpg


In [38]:
df.count()

in_reply_to_user_id       324
text                      333
created_at                333
id                        333
referenced_tweets         333
attachments.media_keys      8
original_text             333
original_tweet            333
media_url                 333
dtype: int64

In [39]:
df.to_excel('btp-011072020T31122020.xlsx',index=False,columns=['id','created_at','referenced_tweets','text','original_tweet','media_url'])